In [93]:
import sisl

struct = sisl.get_sile("/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/RUN.fdf").read_geometry()
struct.write("STRUCT.xyz")

In [96]:
struct.na

4

In [38]:
import numpy as np
struct.xyz.astype(np.float32)

array([[0.     , 0.     , 0.     ],
       [0.     , 2.03915, 2.03915],
       [2.03915, 0.     , 2.03915],
       [2.03915, 2.03915, 0.     ]], dtype=float32)

In [35]:
from ...data import _keys
import numpy as np
structure = {
    _keys.ATOMIC_NUMBERS_KEY: np.array([struct.atoms[i].Z for i in range(struct.na)], dtype=np.int32),
    _keys.PBC_KEY: np.array([True, True, True]) # abacus does not allow non-pbc structure
}
structure[_keys.POSITIONS_KEY] = struct.xyz.astype(np.float32)
structure[_keys.CELL_KEY] = struct.cell.astype(np.float32)

In [36]:
structure

{'atomic_numbers': array([79, 79, 79, 79], dtype=int32),
 'pbc': array([ True,  True,  True]),
 'pos': array([[0.     , 0.     , 0.     ],
        [0.     , 2.03915, 2.03915],
        [2.03915, 0.     , 2.03915],
        [2.03915, 2.03915, 0.     ]], dtype=float32),
 'cell': array([[4.0783, 0.    , 0.    ],
        [0.    , 4.0783, 0.    ],
        [0.    , 0.    , 4.0783]], dtype=float32)}

In [21]:
import sisl
# hamil = sisl.get_sile("/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/RUN.fdf").read_hamiltonian()
hamil =  sisl.Hamiltonian.read('/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/Au_cell.HSX')
basis_siesta = [hamil.atoms[i].orbitals[0].name() for i in range(hamil.na)]
hamil.no

60

In [7]:
import numpy as np
site_norbs = np.array([hamil.atoms[i].no for i in range(hamil.na)])
site_norbs 
site_norbs.cumsum()

array([15, 30, 45, 60])

In [19]:
import sisl
# hamil = sisl.get_sile("/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/RUN.fdf").read_hamiltonian()
DM =  sisl.DensityMatrix.read('/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/Au_cell.DM')
# basis_siesta = [overlap.atoms[i].orbitals[0].name() for i in range(overlap.na)]
# basis_siesta
# overlap.nsc
DM.no

60

In [17]:
DM.nsc

array([5, 5, 5], dtype=int32)

In [18]:
125*60

7500

In [102]:

from collections import Counter
hamil =  sisl.Hamiltonian.read('/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/Au_cell.HSX')
hamil.atoms[0].tag
basis_siesta = {}
basis = {}
for i in range(hamil.na):
    if hamil.atoms[i].tag not in basis_siesta.keys():
        basis_siesta[hamil.atoms[i].tag] = []
        for j in range(hamil.atoms[i].no):
            basis_siesta[hamil.atoms[i].tag].append(hamil.atoms[i].orbitals[j].name())


for atom_type in basis_siesta.keys():
    split_basis = []
    for i in range(len(basis_siesta[atom_type])):
        split_basis.append(list(basis_siesta[atom_type][i])[1])
    
    counted_basis = Counter(split_basis)
    
    counted_basis_list = []
    for basis_type in counted_basis.keys():
        if basis_type == 's':
            counted_basis_list.append(str(int(counted_basis['s']/1))+'s')
        elif basis_type == 'p':
            assert abs(counted_basis['p']%3)<1e-6, "p orbital is not multiple of 3"
            counted_basis_list.append(str(int(counted_basis['p']/3))+'p')
        elif basis_type == 'd':
            assert abs(counted_basis['d']%5)<1e-6, "d orbital is not multiple of 5"
            counted_basis_list.append(str(int(counted_basis['d']/5))+'d')
        elif basis_type == 'f':
            assert abs(counted_basis['f']%7)<1e-6, "f orbital is not multiple of 7"
            counted_basis_list.append(str(int(counted_basis['f']/7))+'f')
    
    basis[atom_type] = "".join(counted_basis_list)
basis

{'Au': '2s1p2d'}

In [111]:
import re

def parse_orbital_string(orbital_string):
    # 定义轨道类型到数字的映射
    orbital_map = {'s': 0, 'p': 1, 'd': 2,'f': 3}
    # 使用正则表达式匹配数字和轨道类型
    pattern = re.compile(r'(\d+)([spdf])')
    
    result = []
    
    for match in pattern.finditer(orbital_string):
        number = int(match.group(1))
        orbital_type = match.group(2)
        result.extend([orbital_map[orbital_type]] * number)
    
    return result

# 示例字符串
orbital_string = '2s1p2d'

# 解析并输出结果
parsed_list = parse_orbital_string(orbital_string)
print(parsed_list)


[0, 0, 1, 2, 2]


In [86]:
path = '/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out'
import os
import re

# def find_and_save_system_label_content(path):
#     # 用于存储包含SystemLabel标签的文件及其内容
#     fdf_files_with_system_label_content = {}

#     # 遍历给定路径及其子目录
#     for root, dirs, files in os.walk(path):
#         for file in files:
#             if file.endswith('.fdf'):
#                 file_path = os.path.join(root, file)
#                 try:
#                     with open(file_path, 'r', encoding='utf-8') as f:
#                         content = f.read()
#                         # 使用正则表达式匹配SystemLabel及其后面的内容
#                         match = re.search(r'\bAtomicCoordinatesAndAtomicSpecies\b\s*(\S+)', content)
#                         if match:
#                             system_label_content = match.group(1)
#                             fdf_files_with_system_label_content[file_path] = system_label_content
#                 except (IOError, UnicodeDecodeError) as e:
#                     print(f"Error reading {file_path}: {e}")

#     return fdf_files_with_system_label_content.keys(), fdf_files_with_system_label_content.values()
def find_system_label(path,str_to_find):
    # 用于存储包含SystemLabel标签的文件及其内容
    
    fdf_files_with_system_label_content = {}
    # 遍历给定路径及其子目录
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.fdf'):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        # 使用正则表达式匹配SystemLabel及其后面的内容
                        match = re.search(r'\b'+str_to_find+r'\b\s*(\S+)', content)
                        if match:
                            system_label_content = match.group(1)
                            fdf_files_with_system_label_content[file_path] = system_label_content
                            break
                except:
                    print(f"don't find {str_to_find} in {file_path}")

    return file_path, system_label_content 
# 示例路径
path = '/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out'

# 查找包含SystemLabel标签的.fdf文件并保存其后面的内容
# result = find_system_label(path, 'AtomicCoordinatesAndAtomicSpecies')
result = find_system_label(path, 'SystemLabel')
print(result)
result[0]

('/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/RUN.fdf', 'Au_cell')


'/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/RUN.fdf'

TypeError: 'dict_keys' object is not subscriptable

In [4]:
import sisl
hamil =  sisl.Hamiltonian.read('/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/Au_cell.HSX')
# eig_fdf = hamil.eigh()

band = sisl.BandStructure(hamil, [[0., 0.,0], 
                                  [0.3750000000,	0.3750000000,	0.7500000000],
                                  [0.5000000000,	0.5000000000,	0.5000000000], 
                                  [0.6250000000,	0.2500000000,	0.6250000000],
                                  [0.5000000000,	0.2500000000,	0.7500000000],
                                  [0.7500000000,	0.2500000000,	0.5000000000],
                                  [0.5000000000,	0.0000000000,	0.5000000000]], [20,20,20,20,20,20],
                                 [r'$\Gamma$', 'K', 'L', 'U','W', 'W2', 'X']) 

eigs = band.apply.array.eigh()
band.k.shape


(121, 3)

In [179]:
import sisl
# hamil = sisl.get_sile("/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/RUN.fdf").read_hamiltonian()
hamil =  sisl.Hamiltonian.read('/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out/Au_cell.HSX')
hamil.shape

(60, 7500, 2)

In [197]:
print(hamil)

Hamiltonian{non-zero: 150156, orthogonal: False,
 Spin{unpolarized, kind=f},
 Geometry{na: 4, no: 60,
  Atoms{species: 1,
   Atom{Au, Z: 79, mass(au): 196.96657, maxR: -1.00000,
    AtomicOrbital{6sZ1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{6sZ2, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{6pyZ1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{6pzZ1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{6pxZ1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{5dxyZ1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{5dyzZ1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{5dz2Z1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{5dxzZ1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{5dx2-y2Z1, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{5dxyZ2, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{5dyzZ2, q0: 0.0, Orbital{R: -1.00000, q0: 0.0}},
    AtomicOrbital{5dz2

In [166]:
import numpy as np
hamil.nsc
central_cell = [int(np.floor(hamil.nsc[i]/2)) for i in range(3)]
Rvec_list = []
for rx in range(central_cell[0],hamil.nsc[0]):
    for ry in range(central_cell[1],hamil.nsc[1]):
        for rz in range(central_cell[2],hamil.nsc[2]):
            Rvec_list.append([rx-central_cell[0],ry-central_cell[1],rz-central_cell[2]])


len(Rvec_list)
Rvec_list

[[0, 0, 0],
 [0, 0, 1],
 [0, 0, 2],
 [0, 1, 0],
 [0, 1, 1],
 [0, 1, 2],
 [0, 2, 0],
 [0, 2, 1],
 [0, 2, 2],
 [1, 0, 0],
 [1, 0, 1],
 [1, 0, 2],
 [1, 1, 0],
 [1, 1, 1],
 [1, 1, 2],
 [1, 2, 0],
 [1, 2, 1],
 [1, 2, 2],
 [2, 0, 0],
 [2, 0, 1],
 [2, 0, 2],
 [2, 1, 0],
 [2, 1, 1],
 [2, 1, 2],
 [2, 2, 0],
 [2, 2, 1],
 [2, 2, 2]]

In [190]:
hamil_csr = hamil.tocsr()
block = {}
for Rvec in Rvec_list:
    off = hamil.geometry.sc_index(Rvec) * hamil.geometry.no
    block[str(Rvec)] = hamil_csr[:,0+off:hamil.geometry.no+off].toarray()

In [196]:
block[str([0,0,2])]

array([[-2.8917704e-09,  4.1865745e-14, -1.9868723e-25, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 4.1865745e-14,  1.3782349e-17,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.9868714e-25,  0.0000000e+00,  3.5500010e-09, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [123]:
hamil[0,0]

array([-6.0882545,  1.       ], dtype=float32)

In [178]:
hamil_csr = hamil.tocsr()
hamil_csr.shape

(60, 7500)

In [133]:
import numpy as np
from scipy.sparse import csr_matrix

# 创建一个稠密矩阵
dense_matrix = np.array([
    [0, 0, 1, 0],
    [2, 0, 0, 0],
    [0, 3, 0, 4],
    [5, 0, 0, 0]
])

# 将稠密矩阵转换为 CSR 格式
sparse_matrix = csr_matrix(dense_matrix)

# 打印 CSR 格式矩阵
print("CSR matrix:\n", sparse_matrix)

# 提取第2行
row_slice = sparse_matrix[2, :]
print("Row slice:\n", row_slice)
print("Row slice (dense format):\n", row_slice.toarray())


CSR matrix:
   (0, 2)	1
  (1, 0)	2
  (2, 1)	3
  (2, 3)	4
  (3, 0)	5
Row slice:
   (0, 1)	3
  (0, 3)	4
Row slice (dense format):
 [[0 3 0 4]]


In [6]:
import numpy as np

def convert_kpoints_bohr_inv_to_twopi_over_a(kpoints_bohr_inv, lattice_const_a_angstrom):
    a0 = 0.529177  # Bohr radius in Å
    scale = lattice_const_a_angstrom / (a0 * 2*np.pi)
    kpoints_twopi_over_a = np.dot(kpoints_bohr_inv, scale)
    return kpoints_twopi_over_a


kpoints_bohr_inv = np.array([
    [0.000000, 0.000000, 0.000000],
    [0.407635, 0.000000, 0.000000],
])

# lattice_const_a_angstrom = np.array([4.0782999992, 4.0782999992, 4.0782999992])  # Lattice constant in Å


lattice_const_a_angstrom = np.array([  [4.0782999992,0.0000000000,0.0000000000],
                                       [0.0000000000,4.0782999992,0.0000000000],
                                       [0.0000000000,0.0000000000,4.0782999992]])

kpoints_twopi_over_a = convert_kpoints_bohr_inv_to_twopi_over_a(kpoints_bohr_inv, lattice_const_a_angstrom)
print("K-points in 2π/a units:\n", kpoints_twopi_over_a)


K-points in 2π/a units:
 [[0.         0.         0.        ]
 [0.49999977 0.         0.        ]]


In [4]:
import os
path = "/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out_withband"
for_Kpt_bands = False
for_system_label = False
str_to_find = 'LatticeVectors'


if for_system_label and for_Kpt_bands:
    raise ValueError("for_system_label and for_Kpt_bands \
                        cannot both be True at the same time.")
file_path = None
system_label_content = None
for root, _, files in os.walk(path):
    for file in files:
            if not for_Kpt_bands and not file.endswith('.fdf'):
                continue
            
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    match = re.search(r'\b'+str_to_find+r'\b\s*(\S+)', content)
                    if match and for_system_label:
                        system_label_content = match.group(1)
                        break
            except:
                print(f"don't find {str_to_find} in {file_path}")

if for_system_label and system_label_content is None:
    print(f"don't find {str_to_find} in {file_path}, use the default value: siesta")

print(file_path)

don't find LatticeVectors in /personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out_withband/RUN_band.fdf
don't find LatticeVectors in /personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out_withband/STRUCT.fdf
/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out_withband/STRUCT.fdf


In [7]:
from dftio.io.siesta.siesta_parser import SiestaParser


root = "/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta"
prefix = "siesta_out_withband"

parser = SiestaParser(root=root, prefix=prefix)

In [8]:
parser.raw_datas

['/personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out_withband']

In [26]:
parser.get_structure(0)

{'atomic_numbers': array([79, 79, 79, 79], dtype=int32),
 'pbc': array([ True,  True,  True]),
 'pos': array([[[0.     , 0.     , 0.     ],
         [0.     , 2.03915, 2.03915],
         [2.03915, 0.     , 2.03915],
         [2.03915, 2.03915, 0.     ]]], dtype=float32),
 'cell': array([[[4.0783, 0.    , 0.    ],
         [0.    , 4.0783, 0.    ],
         [0.    , 0.    , 4.0783]]], dtype=float32)}

In [36]:
parser.get_eigenvalue(0)['eigenvalue'][0, 10, 20]

don't find WELCOME in /personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out_withband/Au_cell.TSHS


-4.189

In [11]:
parser.get_eigenvalue(0)

don't find WELCOME in /personal/DeepTB/dptb_Zjj/DFTIO/dftio/test/data/siesta/siesta_out_withband/Au_cell.TSHS


{'eigenvalue': array([[[-11.9863,  -9.2147,  -9.2147, ...,  62.7709,  62.7709,
           62.7709],
         [-11.9717,  -9.2071,  -9.2071, ...,  62.7298,  62.7552,
           62.7552],
         [-11.9278,  -9.1843,  -9.1843, ...,  62.6078,  62.7076,
           62.7076],
         ...,
         [-11.8987,  -9.1655,  -9.1655, ...,  62.5433,  62.5433,
           62.5433],
         [-11.9643,  -9.2022,  -9.2022, ...,  62.7138,  62.7138,
           62.7138],
         [-11.9863,  -9.2147,  -9.2147, ...,  62.7709,  62.7709,
           62.7709]]], dtype=float32),
 'kpoint': array([[0.        , 0.        , 0.        ],
        [0.02500029, 0.        , 0.02500029],
        [0.05000059, 0.        , 0.05000059],
        [0.07499966, 0.        , 0.07499966],
        [0.09999996, 0.        , 0.09999996],
        [0.12500025, 0.        , 0.12500025],
        [0.15000054, 0.        , 0.15000054],
        [0.17499961, 0.        , 0.17499961],
        [0.19999991, 0.        , 0.19999991],
        [0.225

In [12]:
parser.get_basis(0)

{'Au': '2s1p2d'}

In [61]:
parser.get_blocks(idx=0, hamiltonian=False, overlap=False, density_matrix=True)[2][0]['0_0_0_0_0'][0,0]

FileNotFoundError: Density Matrix file not found.

In [51]:
parser.get_blocks(idx=0, hamiltonian=False, overlap=True, density_matrix=False)[1][0]

{'0_0_0_-1_-1': array([[ 7.35383310e-07,  0.00000000e+00,  1.25149218e-05,
          1.25149218e-05,  0.00000000e+00,  0.00000000e+00,
          5.25982955e-07,  1.51838208e-07,  0.00000000e+00,
         -2.62991477e-07,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -3.00263946e-11,
         -3.00263807e-11,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.25149218e-05,  3.00263980e-11, -1.13848051e-04,
         -1.19959244e-04,  0.00000000e+00,  0.00000000e+00,
         -9.37155164e-06, -3.04278456e-06,  0.00000000e+00,
          4.49094841e-06,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.25149218e-05,  3.00263807e-11, -1.19959244e-04,
         -1.13848051e-

In [39]:
parser.get_blocks(idx=0, hamiltonian=True, overlap=False, density_matrix=False)[0][0]['1_2_0_-1_0'][0,1]

-4.234535e-07

In [24]:
parser.get_blocks(idx=0,hamiltonian=True,overlap=False,density_matrix=False)[0][0]['0_0_0_0_0']

array([[-4.06250906e+00, -2.37229967e+00,  4.76820909e-11,
         6.42628062e-10,  4.76819730e-11,  5.95747549e-11,
         2.97529362e-10, -4.10882928e-09,  2.97518760e-10,
         4.38596132e-10,  3.78797722e-11,  7.23205593e-11,
        -1.59096292e-08,  7.23241259e-11,  1.66258091e-10],
       [-2.37229967e+00,  3.85611868e+00,  2.47695753e-10,
         7.43918038e-10,  2.47692866e-10, -7.58670991e-12,
         2.77096096e-10, -2.33922215e-08,  2.77100315e-10,
         3.38956169e-10, -2.07356216e-11,  2.92250460e-11,
        -4.52482540e-09,  2.92213267e-11,  1.18970930e-10],
       [ 4.76826217e-11,  2.47695309e-10,  3.85423154e-01,
         3.38540501e-10,  1.39611364e-10, -7.05170020e-11,
         3.28115174e-10, -8.21578222e-11,  2.88297786e-15,
        -2.17933879e-10,  1.00248733e-11,  9.21002372e-11,
        -1.06588896e-11,  2.05612441e-15, -2.22133319e-11],
       [ 6.35094088e-10,  7.32930605e-10,  3.38540501e-10,
         3.85423154e-01,  3.38557182e-10, -4.39706688